## Baseline System

### idea

For each question, calculate "how many words (in portion) in the question can be found in the context".

The context is defined as the passage title and the paragraph content.

Further, stop words are eliminated from all texts.

In [1]:
import json
import os
import timeit

import numpy as np
import pandas as pd
import string

from scipy import stats

import nltk

In [2]:
data_dir = ".."
train_path = os.path.join(data_dir, "training.json")
dev_path = os.path.join(data_dir, "development.json")
test_path = os.path.join(data_dir, "testing.json")

In [3]:
df = pd.read_csv("baseline_stats.csv", index_col=0)

## training accuracy

In [4]:
stats.pearsonr(df["ratio"], df["label"])

(0.07579107286924916, 2.563369440365878e-128)

In [57]:
label = df["label"].values

for threshold in range(20):
    threshold /= 20
    
    pred = (df["ratio"].values > threshold).astype(int)
    
    correct = (np.logical_and(pred == label, label == 1)).sum()
    p_total = (pred == 1).sum()
    r_total = (label == 1).sum()
    
    p = correct / p_total
    r = correct / r_total
    f = 2*p*r / (p+r)

#     p_tot = 0
#     r_tot = 0
#     p_cor = 0
#     r_cor = 0
    
#     for i in range(len(df)):
#         ratio = df.loc[i]["ratio"]
#         label = df.loc[i]["label"]
        
#         pred = ratio > threshold
        
#         if pred == 1:
#             p_tot += 1
#             if label == 1:
#                 p_cor += 1
#         if label == 1:
#             r_tot += 1
#             if pred == 1:
#                 r_cor += 1
        
#         if i % 10000 == 0:
#             print(i)
    print(f"{threshold:.2f}: p={p:.5f}, r={r:.5f}, f={f:.5f}")

0.00: p=0.34785, r=0.88662, f=0.49967
0.05: p=0.34786, r=0.88662, f=0.49968
0.10: p=0.34875, r=0.88438, f=0.50024
0.15: p=0.35186, r=0.85166, f=0.49798
0.20: p=0.35483, r=0.74005, f=0.47967
0.25: p=0.35633, r=0.63605, f=0.45676
0.30: p=0.35983, r=0.60439, f=0.45110
0.35: p=0.36184, r=0.47964, f=0.41250
0.40: p=0.36584, r=0.38813, f=0.37666
0.45: p=0.37275, r=0.35701, f=0.36471
0.50: p=0.37600, r=0.21035, f=0.26978
0.55: p=0.37690, r=0.20958, f=0.26937
0.60: p=0.38638, r=0.14323, f=0.20899
0.65: p=0.39229, r=0.13693, f=0.20301
0.70: p=0.39548, r=0.08220, f=0.13610
0.75: p=0.40126, r=0.04522, f=0.08128
0.80: p=0.41554, r=0.03003, f=0.05601
0.85: p=0.44233, r=0.02403, f=0.04558
0.90: p=0.48374, r=0.02066, f=0.03963
0.95: p=0.48466, r=0.02054, f=0.03941


In [24]:
ratio = (df["ratio"].values == 1).astype(int)
(ratio==label).sum() / len(label)

0.6639554030194219

## development accuracy

In [82]:
with open(dev_path) as fin:
    j = json.load(fin)

data = j['data']
    
tokset_stopwords = set(nltk.corpus.stopwords.words("english"))
tokset_punct = set(string.punctuation)

i = 0

d = {}

for passage in data:
    title = passage['title']
    tokset_title = set(nltk.tokenize.word_tokenize(title.lower()))
    paragraphs = passage['paragraphs']
    for paragraph in paragraphs:
        context = paragraph['context']
        tokset_context = set(nltk.tokenize.word_tokenize(context.lower()))
        qas = paragraph['qas']
        for qa in qas:
            question = qa['question']
            tokset_question = set(nltk.tokenize.word_tokenize(question.lower()))
            qid = qa['id']
            tokset_non_stop = tokset_question - tokset_punct - tokset_stopwords
            tokset_not_overlap = tokset_non_stop - tokset_title - tokset_context
            non_stop_count = len(tokset_non_stop)
            not_overlap_count = len(tokset_not_overlap)
            ratio = not_overlap_count/non_stop_count if non_stop_count != 0 else 1
            label = int(qa["is_impossible"])

            d[qid] = int(ratio > 0.50)
            i += 1

            if i % 5000 == 0:
                print(i)

5000
10000
15000
20000
25000


In [83]:
dev_out_path = "dev_out.json"
with open(dev_out_path, "w") as fout:
    json.dump(d, fout)

## gen test output

In [4]:
with open(test_path) as fin:
    j = json.load(fin)

data = j['data']
    
tokset_stopwords = set(nltk.corpus.stopwords.words("english"))
tokset_punct = set(string.punctuation)

i = 0

d = {}

for passage in data:
    title = passage['title']
    tokset_title = set(nltk.tokenize.word_tokenize(title.lower()))
    paragraphs = passage['paragraphs']
    for paragraph in paragraphs:
        context = paragraph['context']
        tokset_context = set(nltk.tokenize.word_tokenize(context.lower()))
        qas = paragraph['qas']
        for qa in qas:
            question = qa['question']
            tokset_question = set(nltk.tokenize.word_tokenize(question.lower()))
            qid = qa['id']
            tokset_non_stop = tokset_question - tokset_punct - tokset_stopwords
            tokset_not_overlap = tokset_non_stop - tokset_title - tokset_context
            non_stop_count = len(tokset_non_stop)
            not_overlap_count = len(tokset_not_overlap)
            ratio = not_overlap_count/non_stop_count if non_stop_count != 0 else 1
#             label = int(qa["is_impossible"])

            d[qid] = int(ratio > 0.1)
            i += 1

            if i % 5000 == 0:
                print(i)

5000
10000


In [5]:
test_out_path = "test_out.json"
with open(test_out_path, "w") as fout:
    json.dump(d, fout)

#### test output to csv

In [6]:
test_out_path = "test_out.json"
with open(test_out_path) as fin:
    j = json.load(fin)
    
df = pd.DataFrame(columns=["Predicted"])
df.index.name = "Id"

for k, v in j.items():
    df.loc[k] = v

df.to_csv("test_out.csv")


#### (DO NOT USE) work-around of `unable to find 17 required key values ...`

In [6]:
kaggle_sample = pd.read_csv("sample.csv", index_col=0)

In [8]:
test_out_path = "test_out.json"
with open(test_out_path) as fin:
    j = json.load(fin)
    
for k in kaggle_sample.index:
    if k not in j:
        j[k] = 0

df = pd.DataFrame(columns=["Predicted"])
df.index.name = "Id"

for k, v in j.items():
    df.loc[k] = v

df.to_csv("test_out.csv")


In [12]:
test_out_path = "test_out.json"
with open(test_out_path) as fin:
    j = json.load(fin)

cnt = 0
for k in kaggle_sample.index:
    if k not in j:
        cnt += 1
        
print(cnt)
print(len(j))

17400
11856


---
## Stats Gen

In [3]:
with open(train_path) as fin:
    j = json.load(fin)

In [4]:
data = j["data"]

In [44]:
df = pd.DataFrame(columns=["id", "non_stop_count", "not_overlap_count", "ratio", "label"])

In [49]:
def f():

    tokset_stopwords = set(nltk.corpus.stopwords.words("english"))
    tokset_punct = set(string.punctuation)
    
    i = 0
    
    for passage in data:
        title = passage['title']
        tokset_title = set(nltk.tokenize.word_tokenize(title.lower()))
        paragraphs = passage['paragraphs']
        for paragraph in paragraphs:
            context = paragraph['context']
            tokset_context = set(nltk.tokenize.word_tokenize(context.lower()))
            qas = paragraph['qas']
            for qa in qas:
                question = qa['question']
                tokset_question = set(nltk.tokenize.word_tokenize(question.lower()))
                qid = qa['id']
                tokset_non_stop = tokset_question - tokset_punct - tokset_stopwords
                tokset_not_overlap = tokset_non_stop - tokset_title - tokset_context
                non_stop_count = len(tokset_non_stop)
                not_overlap_count = len(tokset_not_overlap)
                ratio = not_overlap_count/non_stop_count if non_stop_count != 0 else 1
                label = int(qa["is_impossible"])
                
                df.loc[i] = (qid, non_stop_count, not_overlap_count, ratio, label)
                i += 1
                
                if i % 1000 == 0:
                    print(i)

                # for debug
#                 print(tokset_title)
#                 print(tokset_context)
#                 print(tokset_question)
#                 print(tokset_non_stop)
#                 print(tokset_not_overlap)
#                 print(ratio)
#                 print(label)
#                 return

In [50]:
f()

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


In [51]:
df.to_csv("baseline_stats.csv")

---
## Depricated, DO NOT USE

In [5]:
passage = data[0]

In [6]:
title = passage['title']
paragraphs = passage['paragraphs']

In [7]:
paragraph = paragraphs[0]

In [8]:
context = paragraph['context']
qas = paragraph['qas']

In [9]:
qa = qas[0]

In [10]:
question = qa['question']
qid = qa['id']

In [11]:
import nltk

In [12]:
title

'Beyoncé'

In [13]:
context

'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'

In [14]:
question

'When did Beyoncé release Dangerously in Love?'

In [15]:
toks_title = nltk.tokenize.word_tokenize(title.lower())
toks_context = nltk.tokenize.word_tokenize(context.lower())
toks_question = nltk.tokenize.word_tokenize(question.lower())

In [16]:
print(toks_title)
print(toks_context)
print(toks_question)

['beyoncé']
['beyoncé', 'giselle', 'knowles-carter', '(', '/biːˈjɒnseɪ/', 'bee-yon-say', ')', '(', 'born', 'september', '4', ',', '1981', ')', 'is', 'an', 'american', 'singer', ',', 'songwriter', ',', 'record', 'producer', 'and', 'actress', '.', 'born', 'and', 'raised', 'in', 'houston', ',', 'texas', ',', 'she', 'performed', 'in', 'various', 'singing', 'and', 'dancing', 'competitions', 'as', 'a', 'child', ',', 'and', 'rose', 'to', 'fame', 'in', 'the', 'late', '1990s', 'as', 'lead', 'singer', 'of', 'r', '&', 'b', 'girl-group', 'destiny', "'s", 'child', '.', 'managed', 'by', 'her', 'father', ',', 'mathew', 'knowles', ',', 'the', 'group', 'became', 'one', 'of', 'the', 'world', "'s", 'best-selling', 'girl', 'groups', 'of', 'all', 'time', '.', 'their', 'hiatus', 'saw', 'the', 'release', 'of', 'beyoncé', "'s", 'debut', 'album', ',', 'dangerously', 'in', 'love', '(', '2003', ')', ',', 'which', 'established', 'her', 'as', 'a', 'solo', 'artist', 'worldwide', ',', 'earned', 'five', 'grammy', 'aw

In [25]:
set(nltk.corpus.stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [30]:
import string
set(string.punctuation)

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}